# Turbo Stage Example


In [1]:
import numpy as np
from parafoil import TurboRowPassage, TurboStagePassage, TurboMeshParameters
from ezmesh import visualize_mesh
from parafoil import CamberThicknessAirfoil

rotor_airfoil = CamberThicknessAirfoil(
    inlet_angle=np.radians(-60),
    outlet_angle=np.radians(-51),
    upper_thick_prop=[0.015, 0.05, 0.05  , 0.015],
    lower_thick_prop=[0.015, 0.05, 0.05, 0.015],
    leading_prop=0.5,
    trailing_prop=0.5,
    chord_length=0.018,
)

stator_airfoil = CamberThicknessAirfoil(
    inlet_angle=np.radians(26),
    outlet_angle=np.radians(12),
    upper_thick_prop=[0.015, 0.05, 0.05  , 0.015],
    lower_thick_prop=[0.015, 0.05, 0.05, 0.015],
    leading_prop=0.5,
    trailing_prop=0.5,
    chord_length=0.018,
)

axial_turbo_passage = TurboStagePassage(
    inflow_passage=TurboRowPassage(
        airfoil=rotor_airfoil,
        spacing_to_chord=1.0,
        leading_edge_gap_to_chord=0.25,
        trailing_edge_gap_to_chord=0.25,
        type="camber",
        mesh_params=TurboMeshParameters(
            airfoil_label="wall1",
            bottom_label="periodic1",
            top_label="periodic2",
            inlet_label="inflow",
            outlet_label="outmix",
        )
    ),
    outflow_passage=TurboRowPassage(
        airfoil=stator_airfoil,
        spacing_to_chord=1.0,
        leading_edge_gap_to_chord=0.25,
        trailing_edge_gap_to_chord=0.25,
        type="camber",
        mesh_params=TurboMeshParameters(
            airfoil_label="wall2",
            bottom_label="periodic3",
            top_label="periodic4",
            inlet_label="inmix",
            outlet_label="outflow",
        )
    )
)

# axial_turbo_passage.inflow_passage.visualize()
# axial_turbo_passage.outflow_passage.visualize()

# from ezmesh.exporters import export_to_su2

# meshes = axial_turbo_passage.get_meshes()
# export_to_su2(meshes, "./simulation_out/passage1.su2")
# visualize_mesh(meshes)


In [2]:
from parafoil.optimize import BaseOptimizer
from paraflow import run_simulation, get_flasher, ConfigParameters, get_frames, display_frame

flasher = get_flasher("air", "gas")

optimizer = BaseOptimizer(
    working_directory="./simulation_out",
    passage=axial_turbo_passage,
    config_params=ConfigParameters(
        inlet_total_state=flasher.flash(P=87909, T=276, mach_number=0.4098), 
        target_outlet_static_state=flasher.flash(P=108932, T=296),
        translation=[
            np.array([0.0, 266, 0.0]),
            None
        ]
    ),
    objectives=[("efficiency", "max")]
)

optimizer.optimize()

Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Nurb)
Info    : [ 10%] Meshing curve 2 (Line)
Info    : [ 10%] Meshing curve 3 (Nurb)
Info    : [ 10%] Meshing curve 4 (Line)
Info    : [ 10%] Meshing curve 5 (Line)
Info    : [ 10%] Meshing curve 6 (Line)
Info    : [ 10%] Meshing curve 7 (Line)
Info    : [ 10%] Meshing curve 8 (Line)
Info    : [ 10%] Meshing curve 9 (Line)
Info    : [ 10%] Meshing curve 10 (Line)
Info    : [ 10%] Meshing curve 11 (Line)
Info    : [ 20%] Meshing curve 12 (Line)
Info    : [ 20%] Meshing curve 13 (Line)
Info    : [ 20%] Meshing curve 14 (Line)
Info    : [ 20%] Meshing curve 15 (Line)
Info    : [ 20%] Meshing curve 16 (Line)
Info    : [ 20%] Meshing curve 17 (Line)
Info    : [ 20%] Meshing curve 18 (Line)
Info    : [ 20%] Meshing curve 19 (Line)
Info    : [ 20%] Meshing curve 20 (Line)
Info    : [ 20%] Meshing curve 21 (Line)
Info    : [ 30%] Meshing curve 22 (Line)
Info    : [ 30%] Meshing curve 23 (Line)
Info    : [ 30%] Meshing curve 24 (Line)
I

Exception: ('Problem Error: F can not be set, expected shape (1, 1) but provided (1, 0)', ValueError('cannot reshape array of size 0 into shape (1,1)'))

In [11]:
from paraflow import run_simulation, get_flasher, ConfigParameters, get_frames, display_frame
from paraflow import SimulationResult

flasher = get_flasher("air", "gas")
# sim_results = run_simulation(
#     axial_turbo_passage,
#     config_params=ConfigParameters(
#         inlet_total_state=flasher.flash(P=87909, T=276, mach_number=0.4098), 
#         target_outlet_static_state=flasher.flash(P=108932, T=296),
#         translation=[
#             np.array([0.0, 266, 0.0]),
#             None
#         ]
#     ),
#     working_directory="./simulation_out", 
#     id="1",
#     # auto_delete=True,
#     simulator_config={
#         "custom_download_url": "https://github.com/OpenOrion/SU2/releases/download/7.5.2/SU2-7.5.2-macos64.zip"
#     }
# )
# sim_results.to_file("./simulation_out/cached.pkl")

sim_results = SimulationResult.from_file("./simulation_out/cached.pkl")
sim_results.eval_values
# print(sim_results.eval_values[-1])
# frames = get_frames(
#     sim_results, 
#     property_names=["Pressure"], 
#     num_pnts=500,
# )
# display_frame(frames, "Pressure")
# sim_results.eval_values

KeyError: 'TotTotEff'